[Герман Гаврилов](https://drive.google.com/drive/folders/17-r9k1eEGHcVXzGk87dFqfR6FlNfFAYL?usp=sharing) - моя рабочая папка.

# 7 неделя

In [ ]:
!rm -r sample_data

In [ ]:
# Подключаем google-диск
from google.colab import drive
drive.mount('/content/drive')
my_drv_path = '/content/drive/MyDrive/'

Mounted at /content/drive


## Библиотеки

In [1]:
!pip -q install cohere tiktoken==0.5.1 langchain==0.0.339 openai==1.3.4 faiss-cpu==1.7.4 gspread==3.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.5/220.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [2]:
import os
import re
import timeit
import openai
import zipfile
import getpass
import requests
import tiktoken

from   openai  import OpenAI
from   IPython import display

from   langchain.vectorstores      import FAISS
from   langchain.docstore.document import Document
from   langchain.embeddings.openai import OpenAIEmbeddings
from   langchain.text_splitter     import RecursiveCharacterTextSplitter

import gspread                     # API для работы с Google таблицами
from   google.colab import auth    # модуль для аутентификации
from   google.auth  import default # модуль для работы с учётными данными

from tqdm import tqdm

## Функции и классы

### Функция для ввода OpenAI API-ключа

In [3]:
def get_key_ОpenAI():
  openai.api_key = getpass.getpass('OpenAI API Key:')
  os.environ['OPENAI_API_KEY'] = openai.api_key

### Chat functions

In [4]:
def num_tokens_from_string(string: str) -> int:
  """Возвращает количество токенов в строке."""

  # Выбор кодировщика (`cl100k_base` используется для `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002`)
  encoding = tiktoken.get_encoding('cl100k_base')

  # Разбивка строки на токены и подсчёт из количества
  num_tokens = len(encoding.encode(string))

  return num_tokens
#
#
def split_text(text, verbose=0):
  """Функция разбивает текст на чанки."""

  # Создаём экземпляр сплиттера
  text_splitter = RecursiveCharacterTextSplitter(separators=["##_", "\n\n", "\n", " ", ""],
                                                 chunk_size=1024,
                                                 length_function=lambda x: num_tokens_from_string(x))

  # Получаем список чанков
  source_chunks = []
  for chunk in text_splitter.split_text(text):
    source_chunks.append(Document(page_content=chunk, metadata={}))

  # Обработка чанков
  chank_count = len(source_chunks)
  for number, chank in enumerate(source_chunks):

    # Добавление информации в метаданные чанка о его номере в базе
    chank.metadata['chank'] = f'{number+1}/{chank_count}'

    # Вывод количества слов/токенов во фрагменте, если включен режим verbose
    if verbose:
      count = num_tokens_from_string(chank.page_content)
      print(f"\n Chank#{number+1}/{chank_count}. Tokens in text = {count}\n {'-' * 20}\n{insert_newlines(str(chank))}\n{'=' * 20}")

  # Возвращение списка фрагментов текста
  return source_chunks
#
#
def create_embedding(data, verbose=0):
  """Функция преобразует текстовую базу знаний в векторную."""

  # Разбивка текста на чанки
  source_chunks = split_text(text=data, verbose=verbose)

  # Создание векторной базы знаний на основе чанков
  search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings())

  # Подсчёт общего количества токенов во всех чанках
  count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]))

  # Печать сводной информации по созданию векторной Базы знаний
  print('\n==================== ')
  print('Количество токенов в документе :', count_token)

  # Стоимость эмбеддинга согласно прайса на 22.11.2023 - 0,0001/1К токенов (https://openai.com/pricing#language-models)
  print('ЦЕНА запроса:', 0.0001*(count_token/1000), ' $')
  print('==================== ')

  return search_index
#
#
def load_file(url: str):
  """Функция загрузки документа по url в виде текста."""

  try:
    response = requests.get(url) # получение документа по url
    response.raise_for_status()  # проверка ответа и если была ошибка - формирование исключения
    return response.text

  except Exception as e:
    print(e)
#
#
def load_search_indexes(url: str, verbose=0):
  """Функция загружает текстовую базу знаний и преобразует её в векторную."""

  try:
    return create_embedding(load_file(url), verbose=verbose)

  except Exception as e:
    print(e)
#
#
def insert_newlines(text: str, max_len: int = 120) -> str:
  """Функция форматирует переданный текст по длине для лучшего восприятия на экране."""

  words = text.split()
  lines = []
  current_line = ''

  for word in words:
    if len(current_line + ' ' + word) > max_len:
      lines.append(current_line)
      current_line = ''
    current_line += ' ' + word

  lines.append(current_line)

  return '\n'.join(lines)
#
#
def filtred_docs(docs, limit_score):
  """Функция удаляет из отобранных чанков чанки, у которых score выше значения limit_score.
     При этом limit_score определяет гарантированно ошибочные чанки.
     Далее отбор чанков идёт в зависимости от значения score первого не нулевого score.
     Если есть чанк с score=0, он оставляется единственным.
     Если limit_score = 0, то чанки не фильтруются."""

  if bool(limit_score):
    r = []
    score = 0

    def set_score(d, sc):
      d[0].metadata['score'] = sc
      return d

    for doc in docs:

      s = doc[1]
      if s==0:
        r.append(set_score(doc[0],s))
        break
      if score==0:
        if s<.2: score = s*1.7
        elif s<.3: score = s+.05
        else: score = s+.01
        if score>limit_score: score = limit_score
      if s<score: r.append(set_score(doc,s))

    print(f'Фильр пропустил чанк(ов): {len(r)} из {len(docs)}')

  else:
    r = docs

  return r
#
#
def answer_index(model,
                 system,
                 topic: str,
                 query,
                 search_index,
                 temp=0,
                 verbose_documents=0,
                 verbose_price=0,
                 top_documents=8,
                 limit_score=0.0):
  """Основная функция, которая формирует запрос и получает ответ от OpenAI по заданному вопросу на основе векторной базы знаний."""

  # Выбор варианта вопроса: если есть query, то вопрос задан из группового запроса, и он имеет приоритет
  question = query['question'] if bool(query) else topic

  # Выборка релевантных чанков
  docs = filtred_docs(search_index.similarity_search_with_score(question, k=top_documents), limit_score)

  """Этот блок закоментирован.
     Рассматривается возможность даполнительного запроса с фильтрацией чанков по метаданным.
     Пока нереализовано."""
  #for i, doc in enumerate(docs):
  #  header1 = doc[0].metadata.get('Header 1')
  #  print(f'{i}. {header1}. Score: {doc[1]}')
  #  if header1=='Оценка' or header1=='Экспертиза':
  #    #docs = search_index.similarity_search_with_score(question, k=top_documents,)
  #    #print(f'{i}. {header1}. Score: {doc[1]}')
  #    pass

  message_content = ''            # контекст для GPT
  message_content_display = ''    # контекст для вывода на экран

  for i, doc in enumerate(docs):

    # Формирование контекста для запроса GPT и показа на экран отобранных чанков
    message_content = message_content + f'Отрывок документа №{i+1}:{doc[0].page_content}'
    message_content_display = message_content_display + f"\n Отрывок документа №{i+1}. Chank № {doc[0].metadata.get('chank')}. Score({str(round(doc[1], 3))})\n -----------------------\n{insert_newlines(doc[0].page_content)}\n"

    # Сбор информации для группого запроса
    if bool(query):

      # Выделение из строки метаданных ссылки: если нет - присваиваем пустую строку
      search_link_h1 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 1')) if bool(doc[0].metadata.get('Header 1')) else ''
      search_link_h2 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 2')) if bool(doc[0].metadata.get('Header 2')) else ''
      search_link_h3 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 3')) if bool(doc[0].metadata.get('Header 3')) else ''
      search_link_h4 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 4')) if bool(doc[0].metadata.get('Header 4')) else ''

      # Выбор самой внутренней ссылки: если несколько ссылок, то самая внутренняя ссылается на расположение чанка на сайте
      link = ''
      if bool(search_link_h4): link = search_link_h4.group(1)
      elif bool(search_link_h3): link = search_link_h3.group(1)
      elif bool(search_link_h2): link = search_link_h2.group(1)
      elif bool(search_link_h1): link = search_link_h1.group(1)

      # Заполнение запроса выбранными чанками
      query[f'chank_{i+1}'] = f"Chank № {doc[0].metadata.get('chank')}. Score({str(round(doc[1], 3))}).\n{doc[0].page_content}.\n--------\n{link}"

  # Вывод на экран отобранных чанков
  if (verbose_documents): print(message_content_display)

  user_prompt = '''Ответь на запрос клиента на основе информации представленной в данном документе и текущем диалоге с клиентом.
# Запрос клиента
{topic}
# Текущий диалог с клиентом
```
{dialog}
```
# Документ
```
{doc_chunks}
```'''.format(topic = question, dialog = '', doc_chunks = message_content)


  # Отправка запроса к Open AI
  completion = OpenAI().chat.completions.create(model = model[0],
                                                messages = [{'role': 'system', 'content': system},
                                                            {'role': 'user',   'content': user_prompt}],
                                                temperature = temp)

  # Подсчёт токенов и стоимости
  prompt_tokens = completion.usage.prompt_tokens
  total_tokens = completion.usage.total_tokens
  price_prompt_tokens = prompt_tokens * model[1]/1000
  price_answer_tokens = (total_tokens - prompt_tokens) * model[2]/1000
  price_total_token = price_prompt_tokens + price_answer_tokens

  # Сбор информации для группого запроса
  if bool(query):
    query['price_query'] = price_total_token
    query['price_question'] = price_prompt_tokens
    query['price_answer'] = price_answer_tokens
    query['token_query'] = total_tokens
    query['token_question'] = prompt_tokens
    query['token_answer'] = total_tokens - prompt_tokens

  # Вывод на экран стоимости запроса
  if (verbose_price):
    print('\n======================================================= ')
    print(f'{prompt_tokens} токенов использовано на вопрос. Цена: {round(price_prompt_tokens, 6)} $.')
    print(f'{total_tokens - prompt_tokens} токенов использовано на ответ.  Цена: {round(price_answer_tokens, 6)} $.')
    print(f'{total_tokens} токенов использовано всего.     Цена: {round(price_total_token, 6)} $')
    print('======================================================= ')

  # Ответ OpenAI
  return completion.choices[0].message.content
#
#
def question_normalization(text):
  """Функция нормализует текст вопроса.
     Удаляет лишние пробелы, символы, знаки. Делает первое слово с заглавной буквы."""

  ## Удаление символов "!?.,-" из текста
  #text = re.sub(r'[-!?_]', '', text) # '[-!?.,_]'

  # Разделение текста на слова
  words = text.split()

  ## Проход по каждому слову
  #for i in range(len(words)):
  #
  #  # Если слово состоит из заглавных букв и длина больше 1 символа, считаем это аббревиатурой
  #  if words[i].isupper() and len(words[i]) > 1: continue # пропускаем аббревиатуры
  #  else: words[i] = words[i].lower() # преобразуем в нижний регистр

  ## Преобразуем первое слово в строке к верхнему регистру
  #words[0] = words[0].capitalize()

  # Объединяем слова обратно в строку
  return ' '.join(words)
#
#
def load_text_db(url: str, verbose=0):
  """Функция загружает текстовую базу знаний и преобразует её в векторную."""

  match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
  if match_ is None: response = requests.get(url)
  else: response = requests.get(f'https://docs.google.com/document/d/{match_.group(1)}/export?format=txt')
  response.raise_for_status()

  return create_embedding(response.text, verbose=verbose)
#
#
def load_index_db(name, url: str, verbose=0):
  """Функция загружает векторную базу знаний."""

  # index_db
  name_db = name + '.zip'

  # Скачивание архива базы знаний
  response = requests.get(url) # получение документа по url
  response.raise_for_status()  # проверка ответа, и если была ошибка - формирование исключения

  # Сохранение архива
  with open(name_db, 'wb') as file:
    file.write(response.content)

  # Разархивирование базы знаний
  with zipfile.ZipFile(name_db, 'r') as zip:
    zip.extractall()

  # Загрузка векторной базы знаний
  db = FAISS.load_local(name, OpenAIEmbeddings())

  if verbose:

    docs = db.similarity_search_with_score('', k=10000)
    docs_sorted = sorted(docs, key=lambda x: int(x[0].metadata.get('chank').split('/')[0]))

    for doc in docs_sorted:
      count = num_tokens_from_string(doc[0].page_content)
      print(f"\n Chank#{doc[0].metadata.get('chank')}. Tokens in text = {count}\n {'-' * 20} \n{insert_newlines(str(doc))}\n{'=' * 20}")
    print()

  return db
#
#
def load_db(name, index_db_url: str, text_db_url: str, verbose=0):
  """Функция организует очерёдность загрузки базы знаний.
     Сначала идёт загрузка векторной базы, если она не загружается,
     то загружается база в текстовом формате и потом преобразуется в векторную."""

  try:
    db = load_index_db(name, index_db_url, verbose)
    print('Загрузка векторной базы знаний выполнена успешно.')
    return db

  except Exception as e:
    print('По указанной ссылке векторной базы знаний нет.')
    print(e)
    print('\nИдёт загрузка текстовой базы знаний...')

    try:
      db = load_text_db(text_db_url, verbose)
      print('\nЗагрузка текстовой базы знаний выполнена успешно.')
      return db

    except Exception as e:
      print('\nПо указанной ссылке текстовой базы знаний нет.')
      print(e)
      print('\nОшибка загрузки!')
#
#
def archive_db(folder_to_zip, output_filename, index_db):
  """Создание архива базы знаний (при необходимости).
     В дальнейшем архив нужно поместить на GitHub."""

  # Сохранение папки с векторной базой знаний
  index_db.save_local(folder_to_zip)

  # Архивирование папки с векторной базой знаний
  with zipfile.ZipFile(output_filename, 'w') as zip:
    for root, dirs, files in os.walk(folder_to_zip):
      for file in files:
        zip.write(os.path.join(root, file))
  print(f'Архив базы знаний создан! Имя файла: {output_filename}.')


# Данные по названиям модели и стоимости токена на 22.11.2023 (https://openai.com/pricing#language-models)
# Псевдоним = ['Имя модели', 'Цена токена - вопроса', 'Цена токена - ответа']
MODEL_GPT_4_1106_PREVIEW = ['gpt-4-1106-preview', 0.01,  0.03 ] # 128K tokens
MODEL_GPT_3_5_TURBO_1106 = ['gpt-3.5-turbo-1106', 0.001, 0.002] #  16K tokens

MODEL_COST = {x[0]:x for x in [MODEL_GPT_4_1106_PREVIEW, MODEL_GPT_3_5_TURBO_1106]}

### ChatSheetQuestions

In [5]:
class ChatSheetQuestions:

  # Введём словарь, чтобы можно было обращаться по имени к столбцу
  column_name = {'Вопрос':      0,
                 'Ответ':       1,
                 'Оценка':      2,
                 'Комментарий': 3,
                 'Ошибка':      4,
                 'Модель':      5,
                 'Промпт':      6,
                 'Чанк №1':     7,
                 'Чанк №2':     8,
                 'Чанк №3':     9,
                 'Чанк №4':    10,
                 'Чанк №5':    11,
                 'Чанк №6':    12,
                 'Чанк №7':    13,
                 'Чанк №8':    14}

  top_documents = 8
  temperature = 0
  limit_score = 0
  question_norma = False
  chat_prompt = ''
  index_db = None
  gpt_model = 'gpt-3.5-turbo-1106'

  SELECT_MODEL_GPT = MODEL_COST[gpt_model]


  def __init__(self, url_table, gc_auth) -> None:
    # Название файла с вопросами.
    # Ссылка на общий файл.
    try:

      self.spreadsheet = gc_auth.open_by_url(url_table)
      print(f'Подключились к документу - {self.spreadsheet.title}')

      # Получаем список всех страниц файла
      self.worksheet_list = self.spreadsheet.worksheets()
      print('\nСтраницы документа:')
      print('-------------------')
      for i, worksheet in enumerate(self.worksheet_list):
        print(f'{i}. {worksheet.title}')

    except Exception as e:

      print(e)
      print(f'Ошибка подключения к документу. Проверьте ссылку.')


  def select_sheet(self, number_sheet=1):
    # Выбор страницы в таблице.
    # Устанавливаем номер рабочего листа по списку выше.
    self.number_sheet = number_sheet
    for c in self.column_name: self.column_name[c] = -1

    if not number_sheet==None:

      # Выбор нужного листа из списка
      self.worksheet = self.worksheet_list[number_sheet]
      worksheet = self.worksheet

      # Проверка текущей страницы
      print(f'Текущая страница - "{worksheet.title}"\n')

      # Список всех столбцов на странице
      print('-----№---------Название---')
      for i, col in enumerate(worksheet.row_values(1)):
        print(f'Колонка №{i+1:<2} {col}')
        self.column_name[col] = i

    else: print('Введите номер страницы.')


  def load_sheet(self):
    # Загрузка вопросов.

    column_question = self.column_name['Вопрос']
    #print(f'column_question = {column_question}')
    verbose_question = 0 # @param {type:"integer"} #

    worksheet = self.worksheet
    # В данном блоке происходит заполнение списка запросов вопросами и дополнительной
    # информацией с выбранного листа. Загрузка выполняется ВСЕХ строк с вопросами для
    # последующей корректной работы. Далее из этого списка выбирается нужный диапазон
    # вопросов для группового запроса к GPT.

    # Выбор всех вопросов
    column = worksheet.col_values(column_question+1)
    #print(f'column = {column}')

    if self.question_norma: print('Вопросы будут нормализованы.')

    # Создаём пустой список запросов
    list_query = []
    column_name = self.column_name
    r_question  = column_name['Вопрос']
    r_appraisal = column_name['Оценка']

    # Заполнение списка запросов информацией с выбранного листа
    for i in range(len(column)-1):

      row = worksheet.row_values(i+2)

      # Считывание ячеек с контролем их наличия для избежании ошибки чтения
      # Проверить индексы row со списком в ячейке выше
      question = row[r_question] if len(row)>r_question else ''    # вопрос
      appraisal = row[r_appraisal] if len(row)>r_appraisal else '' # оценка

      # Нормализация вопроса
      question = question_normalization(question) if self.question_norma else question


      # Словарь запроса
      query = {
                'line':           i+2,              # номер строки в документе: берём строки с вопросами
                'question':       question,         # вопрос
                'answer_gpt':     '',               # ответ GPT
                'appraisal':      appraisal,        # оценка ответа
                'bug':            '',               # ошибка
                'comments':       '',               # комментарии
                'chank_1':        '',               # чанк №1
                'chank_2':        '',               # чанк №2
                'chank_3':        '',               # чанк №3
                'chank_4':        '',               # чанк №4
                'chank_5':        '',               # чанк №5
                'chank_6':        '',               # чанк №6
                'chank_7':        '',               # чанк №7
                'chank_8':        '',               # чанк №8
                'gpt_model':      self.gpt_model,   # модель GPT
                'chat_prompt':    self.chat_prompt, # промпт
                'price_query':    0,                # стоимость запроса общая
                'price_question': 0,                # стоимость вопроса с контекстом
                'price_answer':   0,                # стоимость ответа
                'token_query':    0,                # количество токенов всего: вопрос-ответ
                'token_question': 0,                # количество токенов в вопросе с контекстом
                'token_answer':   0,                # количество токенов в ответе
      }
      list_query.append(query)

    # Вывод вопросов
    if verbose_question:
      for query in list_query:
        print(f'Строка №{query["line"]}. Вопрос: {query["question"]}')

    # Диапазон строк с вопросами на выбранной странице
    print(f'\n\nЗагрузка списка вопросов завершена. Количество вопросов: {len(column)-1}.')
    print(f"Диапазон номеров строк с вопросами (включительно): [{list_query[0]['line']}:{list_query[-1]['line']}].")
    self.list_query = list_query
    #print(list_query)


  def execute(self,  row_first = 2, row_end = 2, verbose_answer = 0, verbose_chank = 0):

    # @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: true }

    # Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).

    # Если подвис ответ от GPT можно прервать обработку, установить row_first
    # в значение на которой была прервана обработка и запустить обработку заново.
    # Ранее обработанные вопросы будут сохранены в файле.

    # Обнуление общих затрат на групповой запрос
    total_price_query, total_token_query, total_query = 0, 0, 0

    # Фиксация времени
    start_group = timeit.default_timer()

    list_query = self.list_query
    for query in list_query[row_first-2:row_end-1]:

      # Проверка на пустой вопрос: если пустой - пропуск цикла
      if not bool(query['question'].strip()):
        print(f'На строке № {query["line"]} - вопроса нет.')
        continue

      # Отправка запроса, фиксация времени
      start = timeit.default_timer()
      try:
        query['answer_gpt'] = answer_index(model = self.SELECT_MODEL_GPT,
                                           system = self.chat_prompt,
                                           topic = '',
                                           query = query,
                                           search_index = self.index_db,
                                           temp = self.temperature,
                                           verbose_documents = 0,
                                           verbose_price = 0,
                                           top_documents = self.top_documents,
                                           limit_score = self.limit_score)
        total_price_query += query['price_query']
        total_token_query += query['token_query']
        total_query +=1
      except Exception as e:
        print(f'Ошибка ответа GPT на строке №{query["line"]}. - {e}')
      end = timeit.default_timer()

      # Сообщение об успешности ответа от GPT
      print(f'Строка №{query["line"]}. Ответ на вопрос получен за - {round(end-start, 3)} сек.')

      # Запись ответа в файл
      worksheet = self.worksheet
      r_answer_gpt = self.column_name['Ответ GPT']
      r_gpt_model = self.column_name['Модель GPT']
      r_prompt = self.column_name['Промпт']
      r_chank_1 = self.column_name['Чанк №1']
      r_chank_2 = self.column_name['Чанк №2']
      r_chank_3 = self.column_name['Чанк №3']
      r_chank_4 = self.column_name['Чанк №4']
      r_chank_5 = self.column_name['Чанк №5']
      r_chank_6 = self.column_name['Чанк №6']
      r_chank_7 = self.column_name['Чанк №7']
      r_chank_8 = self.column_name['Чанк №8']

      def update_cell(row, name):
        worksheet.update_cell(query['line'], row+1, query[name]) # ответ GPT

      try:
        if bool(query['answer_gpt']):
          update_cell(r_answer_gpt, 'answer_gpt') # ответ GPT
          update_cell(r_gpt_model, 'gpt_model')   # модель GPT
          update_cell(r_prompt, 'chat_prompt')    # промпт
          update_cell(r_chank_1, 'chank_1')       # чанк №1
          update_cell(r_chank_2, 'chank_2')       # чанк №2
          update_cell(r_chank_3, 'chank_3')       # чанк №3
          update_cell(r_chank_4, 'chank_4')       # чанк №4
          update_cell(r_chank_5, 'chank_5')       # чанк №5
          update_cell(r_chank_6, 'chank_6')       # чанк №6
          update_cell(r_chank_7, 'chank_7')       # чанк №7
          update_cell(r_chank_8, 'chank_8')       # чанк №8

          print(f'Строка №{query["line"]}. Ответ записан в файл.')

          # Контроль номализации вопроса и перезапись его в таблице
          if self.question_norma:
            question_old = worksheet.row_values(query['line'])[self.column_question]
            if not (question_old == query['question']):
              print("    Вопрос нормализован и перезаписан в таблице.")
              print(f"    Старый вариант: {question_old}")
              print(f"    Новый вариант : {query['question']}")
              worksheet.update_cell(query['line'], self.column_question+1, query['question'])
      except Exception as e:
        print('\n========================================================')
        print(f'!!!Ошибка записи строки №{query["line"]} в файл. - {e}')
        print('========================================================')

    end_group = timeit.default_timer()

    # Вывод вопросов и ответов
    if verbose_answer:
      for i, query in enumerate(list_query):
        if query['answer_gpt']:
          print()
          print(insert_newlines(f"ВОПРОС №{i+2}: {query['question']}"))
          print('---------------------------')
          print(insert_newlines(f"\nОТВЕТ: {query['answer_gpt']}"))
          print('===========================\n')
          if verbose_chank:
            print(query['chank_1'])
            print()
            print(query['chank_2'])
            print()
            print(query['chank_3'])
            print()
            print(query['chank_4'])
            print()
            print(query['chank_5'])
            print()
            print(query['chank_6'])
            print()
            print(query['chank_7'])
            print()
            print(query['chank_8'])

    print()
    print('-------------------------------------------')
    print(f'Вопросов в пакетной обработке - {total_query} шт.')
    print(f'Время пакетной обработки      - {round(end_group-start_group, 1)} сек.')
    print(f'Стоимость пакетной обработки  - {round(total_price_query, 4)} $.')
    print(f'Токенов в пакетной обработке  - {total_token_query} шт.')

### FaissDB

In [ ]:
"""
# Список папок с векторными базами (имена папок)
index_db_list = ['DataScience', 'Trading', 'PyTourch', 'AutoML']

class FaissDB:
  @staticmethod
  def create_search_index(db_path,       # путь к папкам с базой FAISS
                          index_db_list, # список папок векторными базами (имена папок)
                          embeddings = OpenAIEmbeddings()):
    flag = True
    for index_db in index_db_list:
      index_db_path = os.path.join(db_path, index_db)
      # count_base = 0
      for curr_base in tqdm(os.listdir(index_db_path)):
        if flag:
          db = FAISS.load_local(os.path.join(index_db_path, curr_base), embeddings)
          flag = False
        else:
          db.merge_from(FAISS.load_local(os.path.join(index_db_path, curr_base), embeddings))
        # count_base += 1
    return db
"""

In [6]:
def load_selected_bases(index_db_links,                           # словарь ссылок на векторные базы
                        selected = ['TelegramPosts_index_db',     # выбранные базы для загрузки
                                    'TelegramEther_index_db',
                                    'TelegramVideo_index_db',
                                    'BusinessMarathon_index_db',
                                    'YouTube_index_db',
                                    'Platrum_index_db']):
  flag = True
  # count_base = 0
  for curr_base in tqdm(selected):
    if flag:
      db = load_db(curr_base, index_db_links[curr_base], 'https://', verbose=0)
      flag = False
    else:
      db.merge_from(load_db(curr_base, index_db_links[curr_base], 'https://', verbose=0))
    # count_base += 1
  return db

## Ввод API ключа

In [7]:
get_key_ОpenAI()

OpenAI API Key:··········


## Создание векторной базы знаний

### Посты в Telegram

In [ ]:
# @title Загрузка базы знаний из постов в Telegram по ссылке в текстовом формате

# Ссылка на базу знаний в текстовом формате
text_db_link = 'https://docs.google.com/document/d/1u01H0WKjgYDJV2-JtLm_Kgd-1hGRKTOhwCv935IFcsk/edit?usp=sharing' #@param string

# Загрузка базы знаний
TelegramPosts_index_db = load_db('TelegramPosts_index_db', 'https://', text_db_link, verbose=0)

По указанной ссылке векторной базы знаний нет.
Invalid URL 'https://': No host supplied

Идёт загрузка текстовой базы знаний...

Количество токенов в документе : 129430
ЦЕНА запроса: 0.012943000000000001  $

Загрузка текстовой базы знаний выполнена успешно.


In [ ]:
# Архивируем полученную базу знаний в векторном формате
archive_db('TelegramPosts_index_db', 'TelegramPosts_index_db.zip', TelegramPosts_index_db)

Архив базы знаний создан! Имя файла: TelegramPosts_index_db.zip.


### Эфир в Telegram

In [ ]:
# @title Загрузка базы знаний из эфира в Telegram по ссылке в текстовом формате

# Ссылка на базу знаний в текстовом формате
text_db_link = 'https://docs.google.com/document/d/1oBYFj46MHr8qZQWKiyWsX8tO-5Wnw33Rxf2nweBp0dc/edit?usp=sharing' #@param string

# Загрузка базы знаний
TelegramEther_index_db = load_db('TelegramEther_index_db', 'https://', text_db_link, verbose=0)

По указанной ссылке векторной базы знаний нет.
Invalid URL 'https://': No host supplied

Идёт загрузка текстовой базы знаний...

Количество токенов в документе : 33901
ЦЕНА запроса: 0.0033901000000000005  $

Загрузка текстовой базы знаний выполнена успешно.


In [ ]:
# Архивируем полученную базу знаний в векторном формате
archive_db('TelegramEther_index_db', 'TelegramEther_index_db.zip', TelegramEther_index_db)

Архив базы знаний создан! Имя файла: TelegramEther_index_db.zip.


### Видео в Telegram

In [ ]:
# @title Загрузка базы знаний из видео в Telegram по ссылке в текстовом формате

# Ссылка на базу знаний в текстовом формате
text_db_link = 'https://docs.google.com/document/d/1WJe4VlYsk-AiXqBY71xunSRU6Mdlbmb7YEq9Rl10Z7U/edit?usp=sharing' #@param string

# Загрузка базы знаний
TelegramVideo_index_db = load_db('TelegramVideo_index_db', 'https://', text_db_link, verbose=0)

По указанной ссылке векторной базы знаний нет.
Invalid URL 'https://': No host supplied

Идёт загрузка текстовой базы знаний...

Количество токенов в документе : 4080
ЦЕНА запроса: 0.00040800000000000005  $

Загрузка текстовой базы знаний выполнена успешно.


In [ ]:
# Архивируем полученную базу знаний в векторном формате
archive_db('TelegramVideo_index_db', 'TelegramVideo_index_db.zip', TelegramVideo_index_db)

Архив базы знаний создан! Имя файла: TelegramVideo_index_db.zip.


### Бизнес-марафон

In [ ]:
# @title Загрузка базы знаний из видео Бизнес-марафона по ссылке в текстовом формате

# Ссылка на базу знаний в текстовом формате
text_db_link = 'https://docs.google.com/document/d/1m9VuLY8Sua1JRqcHh4dzEERmrbuaQw4DEDGZa5YA6K0/edit?usp=sharing' #@param string

# Загрузка базы знаний
BusinessMarathon_index_db = load_db('BusinessMarathon_index_db', 'https://', text_db_link, verbose=0)

По указанной ссылке векторной базы знаний нет.
Invalid URL 'https://': No host supplied

Идёт загрузка текстовой базы знаний...

Количество токенов в документе : 50737
ЦЕНА запроса: 0.0050737  $

Загрузка текстовой базы знаний выполнена успешно.


In [ ]:
# Архивируем полученную базу знаний в векторном формате
archive_db('BusinessMarathon_index_db', 'BusinessMarathon_index_db.zip', BusinessMarathon_index_db)

Архив базы знаний создан! Имя файла: BusinessMarathon_index_db.zip.


### YouTube

In [ ]:
# @title Загрузка базы знаний из видео на YouTube по ссылке в текстовом формате

# Ссылка на базу знаний в текстовом формате
text_db_link = 'https://docs.google.com/document/d/1Ri73WrvpAwBJO-voriz6tG-7xitcLQBwke8pAC8X3H0/edit?usp=sharing' #@param string

# Загрузка базы знаний
YouTube_index_db = load_db('YouTube_index_db', 'https://', text_db_link, verbose=0)

По указанной ссылке векторной базы знаний нет.
Invalid URL 'https://': No host supplied

Идёт загрузка текстовой базы знаний...

Количество токенов в документе : 356384
ЦЕНА запроса: 0.0356384  $

Загрузка текстовой базы знаний выполнена успешно.


In [ ]:
# Архивируем полученную базу знаний в векторном формате
archive_db('YouTube_index_db', 'YouTube_index_db.zip', YouTube_index_db)

Архив базы знаний создан! Имя файла: YouTube_index_db.zip.


## Загрузка векторной базы знаний

In [8]:
# Словарь ссылок на векторные базы
index_db_links = {'All_index_db':              'https://github.com/terrainternship/GPT_Gavrilov_g/raw/main/knowledge_base/BASE_FAISS/faiss_db_db_data_from_all_data_txt.zip',
                  'TelegramPosts_index_db':    'https://github.com/terrainternship/GPT_Gavrilov_g/raw/main/knowledge_base/BASE_FAISS/faiss_db_data_from_txt_telegram_channel.zip',
                  'TelegramEther_index_db':    'https://github.com/terrainternship/GPT_Gavrilov_g/raw/main/knowledge_base/BASE_FAISS/faiss_db_data_from_txt_video_podcast_telegram.zip',
                  'TelegramVideo_index_db':    'https://github.com/terrainternship/GPT_Gavrilov_g/raw/main/knowledge_base/BASE_FAISS/faiss_db_data_from_txt_video_telegram.zip',
                  'BusinessMarathon_index_db': 'https://github.com/terrainternship/GPT_Gavrilov_g/raw/main/knowledge_base/BASE_FAISS/faiss_db_data_from_txt_video_training_marathon.zip',
                  'YouTube_index_db':          'https://github.com/terrainternship/GPT_Gavrilov_g/raw/main/knowledge_base/BASE_FAISS/faiss_db_data_from_txt_youtobe.zip',
                  'Platrum_index_db':          'https://github.com/terrainternship/GPT_Gavrilov_g/raw/main/knowledge_base/BASE_FAISS/faiss_db_db_data_from_txt_platrum.zip'}

In [ ]:
# Загрузка векторной базы знаний
index_db = load_selected_bases(index_db_links)

 17%|█▋        | 1/6 [00:01<00:05,  1.01s/it]

Загрузка векторной базы знаний выполнена успешно.


 33%|███▎      | 2/6 [00:01<00:03,  1.09it/s]

Загрузка векторной базы знаний выполнена успешно.


 50%|█████     | 3/6 [00:02<00:02,  1.18it/s]

...

Загрузка векторной базы знаний выполнена успешно.


 83%|████████▎ | 5/6 [00:04<00:01,  1.01s/it]

Загрузка векторной базы знаний выполнена успешно.


100%|██████████| 6/6 [00:06<00:00,  1.09s/it]

Загрузка векторной базы знаний выполнена успешно.


In [9]:
# Загрузка векторной базы знаний
index_db = load_selected_bases(index_db_links, ['All_index_db'])

100%|██████████| 1/1 [00:01<00:00,  1.81s/it]

Загрузка векторной базы знаний выполнена успешно.


## Тестирование вопросов

In [10]:
auth.authenticate_user()      # аутентифицируем текущего пользователя Colab
creds, _ = default()          # создаём объект учётных данных на основе аутентификации
gc = gspread.authorize(creds) # создаём клиента для таблиц на основе учётных данных

In [11]:
url_table = 'https://docs.google.com/spreadsheets/d/13Z73lWK8bnOHtHnY9ChaUeou-b9unryV4JP87cKJuS4/edit?usp=sharing' #@param string

In [12]:
g = ChatSheetQuestions(url_table, gc)

Подключились к документу - Тестирование БЗ German Gavrilov

Страницы документа:
-------------------
0. Стажёры
1. Аникеев
2. Чижов
3. Лучников


### Тестирование вопросов от Владислава Чижова

In [13]:
MODEL_GPT = 'gpt-3.5-turbo-1106' # @param ['gpt-3.5-turbo-1106', 'gpt-4-1106-preview']
top_documents = 8                # @param {type:'integer'} # Количество отобранных чанков
temperature = 0                  # @param float            # Вариативность ответа
limit_score = 0                  # @param float            # Отфильтровывать чанки при поиске выше этого значения, если '0' - не фильтровать
question_norma = False           # @param {type:'boolean'} # Нормализовать вопрос - Удаляет лишние пробелы из вопроса
max_characters = 4096            # @param {type:'integer'}
max_tokens = 2048                # @param {type:'integer'}
url_prompt = 'https://raw.githubusercontent.com/terrainternship/GPT_Gavrilov_g/main/tgbot_alex/settings/system_prompt.txt' # @param {type:'string'}
chat_prompt = load_file(url_prompt).format(max_tokens = max_tokens, max_characters = max_characters)

In [14]:
g.chat_prompt = chat_prompt
g.top_documents = top_documents
g.temperature = temperature
g.limit_score = limit_score
g.question_norma = question_norma
g.index_db = index_db
g.gpt_model = MODEL_GPT
g.SELECT_MODEL_GPT = MODEL_COST[g.gpt_model]
g.select_sheet(2)

Текущая страница - "Чижов"

-----№---------Название---
Колонка №1  Вопрос
Колонка №2  Ответ GPT
Колонка №3  Оценка
Колонка №4  Комментарий
Колонка №5  Ошибка
Колонка №6  Модель GPT
Колонка №7  Промпт
Колонка №8  Чанк №1
Колонка №9  Чанк №2
Колонка №10 Чанк №3
Колонка №11 Чанк №4
Колонка №12 Чанк №5
Колонка №13 Чанк №6
Колонка №14 Чанк №7
Колонка №15 Чанк №8


In [15]:
# @title Промпт
g.chat_prompt

'# Основные инструкции\nТы консультант по вопросам, связанным с систематизацией бизнеса и деятельностью Германа Гаврилова, основывайся исключительно на информации, представленной в данном документе и текущем диалоге с клиентом. Не добавляй никакой информации из других источников и не используй внешние данные. Строго придерживайся только тех данных, которые содержатся в документе и текущем диалоге. Не упоминай клиенту о наличии документа с информацией и не давай понять, что твои ответы основаны на этом документе. Если информации в предоставленном документе и текущем диалоге недостаточно для ответа на вопрос клиента, сообщи об этом прямо, не пытаясь дополнять ответ внешними данными. Твоя задача - предоставлять точные и конкретные ответы, основанные исключительно на представленных данных. Если запрос клиента выходит за рамки тематики бизнес-систематизации или данных, содержащихся в диалоге и документе, объясни это клиенту, указав на отсутствие необходимой информации для ответа. Следуй инс

In [16]:
# @title Загрузка вопросов
g.load_sheet()



Загрузка списка вопросов завершена. Количество вопросов: 181.
Диапазон номеров строк с вопросами (включительно): [2:182].


In [ ]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 2      # @param {type:'integer'}
row_end = 2        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №2. Ответ на вопрос получен за - 4.528 сек.
Строка №2. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 5.9 сек.
Стоимость пакетной обработки  - 0.0075 $.
Токенов в пакетной обработке  - 7322 шт.


In [ ]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 3      # @param {type:'integer'}
row_end = 3        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №3. Ответ на вопрос получен за - 9.045 сек.
Строка №3. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 12.3 сек.
Стоимость пакетной обработки  - 0.0081 $.
Токенов в пакетной обработке  - 7958 шт.


In [ ]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 4      # @param {type:'integer'}
row_end = 4        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №4. Ответ на вопрос получен за - 7.228 сек.
Строка №4. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 10.1 сек.
Стоимость пакетной обработки  - 0.0071 $.
Токенов в пакетной обработке  - 6897 шт.


In [18]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 5      # @param {type:'integer'}
row_end = 181        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №5. Ответ на вопрос получен за - 9.582 сек.
Строка №5. Ответ записан в файл.
Строка №6. Ответ на вопрос получен за - 6.745 сек.
Строка №6. Ответ записан в файл.
Строка №7. Ответ на вопрос получен за - 6.055 сек.
Строка №7. Ответ записан в файл.
Строка №8. Ответ на вопрос получен за - 5.995 сек.
Строка №8. Ответ записан в файл.
Строка №9. Ответ на вопрос получен за - 11.915 сек.
Строка №9. Ответ записан в файл.
Строка №10. Ответ на вопрос получен за - 6.214 сек.
Строка №10. Ответ записан в файл.
Строка №11. Ответ на вопрос получен за - 10.047 сек.
Строка №11. Ответ записан в файл.
Строка №12. Ответ на вопрос получен за - 15.291 сек.
Строка №12. Ответ записан в файл.
Строка №13. Ответ на вопрос получен за - 10.204 сек.
Строка №13. Ответ записан в файл.
Строка №14. Ответ на вопрос получен за - 8.037 сек.
Строка №14. Ответ записан в файл.
Строка №15. Ответ на вопрос получен за - 8.121 сек.
Строка №15. Ответ записан в файл.
Строка №16. Ответ на вопрос получен за - 7.888 сек.
Строка №

In [17]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 182      # @param {type:'integer'}
row_end = 182        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №182. Ответ на вопрос получен за - 12.668 сек.
Строка №182. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 16.1 сек.
Стоимость пакетной обработки  - 0.0076 $.
Токенов в пакетной обработке  - 7278 шт.


### Тестирование вопросов от Евгения Лучникова

In [19]:
MODEL_GPT = 'gpt-3.5-turbo-1106' # @param ['gpt-3.5-turbo-1106', 'gpt-4-1106-preview']
top_documents = 8                # @param {type:'integer'} # Количество отобранных чанков
temperature = 0                  # @param float            # Вариативность ответа
limit_score = 0                  # @param float            # Отфильтровывать чанки при поиске выше этого значения, если '0' - не фильтровать
question_norma = False           # @param {type:'boolean'} # Нормализовать вопрос - Удаляет лишние пробелы из вопроса
max_characters = 4096            # @param {type:'integer'}
max_tokens = 2048                # @param {type:'integer'}
url_prompt = 'https://raw.githubusercontent.com/terrainternship/GPT_Gavrilov_g/main/tgbot_alex/settings/system_prompt.txt' # @param {type:'string'}
chat_prompt = load_file(url_prompt).format(max_tokens = max_tokens, max_characters = max_characters)

In [20]:
g.chat_prompt = chat_prompt
g.top_documents = top_documents
g.temperature = temperature
g.limit_score = limit_score
g.question_norma = question_norma
g.index_db = index_db
g.gpt_model = MODEL_GPT
g.SELECT_MODEL_GPT = MODEL_COST[g.gpt_model]
g.select_sheet(3)

Текущая страница - "Лучников"

-----№---------Название---
Колонка №1  Вопрос
Колонка №2  Ответ GPT
Колонка №3  Оценка
Колонка №4  Комментарий
Колонка №5  Ошибка
Колонка №6  Модель GPT
Колонка №7  Промпт
Колонка №8  Чанк №1
Колонка №9  Чанк №2
Колонка №10 Чанк №3
Колонка №11 Чанк №4
Колонка №12 Чанк №5
Колонка №13 Чанк №6
Колонка №14 Чанк №7
Колонка №15 Чанк №8


In [21]:
# @title Промпт
g.chat_prompt

'# Основные инструкции\nТы консультант по вопросам, связанным с систематизацией бизнеса и деятельностью Германа Гаврилова, основывайся исключительно на информации, представленной в данном документе и текущем диалоге с клиентом. Не добавляй никакой информации из других источников и не используй внешние данные. Строго придерживайся только тех данных, которые содержатся в документе и текущем диалоге. Не упоминай клиенту о наличии документа с информацией и не давай понять, что твои ответы основаны на этом документе. Если информации в предоставленном документе и текущем диалоге недостаточно для ответа на вопрос клиента, сообщи об этом прямо, не пытаясь дополнять ответ внешними данными. Твоя задача - предоставлять точные и конкретные ответы, основанные исключительно на представленных данных. Если запрос клиента выходит за рамки тематики бизнес-систематизации или данных, содержащихся в диалоге и документе, объясни это клиенту, указав на отсутствие необходимой информации для ответа. Следуй инс

In [22]:
# @title Загрузка вопросов
g.load_sheet()



Загрузка списка вопросов завершена. Количество вопросов: 153.
Диапазон номеров строк с вопросами (включительно): [2:154].


In [ ]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 2      # @param {type:'integer'}
row_end = 2        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №2. Ответ на вопрос получен за - 6.691 сек.
Строка №2. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 8.0 сек.
Стоимость пакетной обработки  - 0.0087 $.
Токенов в пакетной обработке  - 8348 шт.


In [ ]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 3      # @param {type:'integer'}
row_end = 3        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №3. Ответ на вопрос получен за - 18.634 сек.
Строка №3. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 20.6 сек.
Стоимость пакетной обработки  - 0.0078 $.
Токенов в пакетной обработке  - 7049 шт.


In [23]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 4      # @param {type:'integer'}
row_end = 154        # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №4. Ответ на вопрос получен за - 24.439 сек.
Строка №4. Ответ записан в файл.
Строка №5. Ответ на вопрос получен за - 20.312 сек.
Строка №5. Ответ записан в файл.
Строка №6. Ответ на вопрос получен за - 10.98 сек.
Строка №6. Ответ записан в файл.
Строка №7. Ответ на вопрос получен за - 13.061 сек.
Строка №7. Ответ записан в файл.
Строка №8. Ответ на вопрос получен за - 15.052 сек.
Строка №8. Ответ записан в файл.
Строка №9. Ответ на вопрос получен за - 13.598 сек.
Строка №9. Ответ записан в файл.
Строка №10. Ответ на вопрос получен за - 11.9 сек.
Строка №10. Ответ записан в файл.
Строка №11. Ответ на вопрос получен за - 18.245 сек.
Строка №11. Ответ записан в файл.
Строка №12. Ответ на вопрос получен за - 20.455 сек.
Строка №12. Ответ записан в файл.
Строка №13. Ответ на вопрос получен за - 19.391 сек.
Строка №13. Ответ записан в файл.
Строка №14. Ответ на вопрос получен за - 22.339 сек.
Строка №14. Ответ записан в файл.
Строка №15. Ответ на вопрос получен за - 22.833 сек.
Строк